In [ ]:
import numpy as np
import pandas as pd
from collections import *
import torch
import torch.nn as nn
from Models.TwoLayerNet import twoLayerNet 
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 14.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 1

%aimport Models.Solver, Models.ganConv 

In [ ]:
# Generate data for 6 classes:
# Letters
letters = {}

# Fingers: Index, Middle, Ring, Little
fingers = ['index','middle','ring','little']

# Joints
'''
Angles for fingers: 
    - MCP_fe: metacarpophalangeal flexion/extension 
    - MCP_aa: metacarpophalangeal abduction/adduction
    - PIP: Proximal-interphalangeal 

Angles for thumb:
    - TMC_fe: trapeziometacarpal flexion/extension
    - TMC_aa: trapeziometacarpal abduction/adduction
    - MCP_fe: metacarpophalangeal flexion/extension
'''
angles = {'MCP_fe','MCP_aa','PIP'}
angles_thumb = {'TMC_fe','TMC_aa','MCP_fe'}


# Initialize angles to 0 (Rest position of the hand)
default_params = defaultdict(dict)

for finger in fingers:
    for angle in angles:
        default_params[finger][angle] = 0
# Thumb
for angle in angles_thumb:
    default_params['thumb'][angle]=0

# Create a dataframe from default_parameters
df = pd.DataFrame.from_dict(default_params)
print(df)

# Transform the dataframe to a row vector
array = df.as_matrix().ravel()
array = array[~np.isnan(array)]

# Number of parameters
n_params = len(array)

# Parametrize the letters

# Letter A
A = df.copy()
A.loc['MCP_fe':'PIP','index':'ring']=90
A.loc['TMC_aa','thumb']=-45
letters['A']=A

# Letter B
B = df.copy()
B.loc['TMC_fe','thumb']=180
letters['B']=B

# Letter D
D = df.copy()
D.loc['MCP_fe','little':'thumb']=45
D.loc['PIP','little':'ring']=60
D.loc['TMC_fe','thumb'] = 90
letters['D']=D

# Letter E
E = df.copy()
E.loc['MCP_fe','index':'thumb']=90
E.loc['PIP','index':'ring']=90
E.loc['TMC_aa','thumb']=-90
letters['E']=E

# Letter F
F = df.copy()
F.loc['MCP_fe','index']=30
F.loc['PIP','index']=90
F.loc['TMC_fe','thumb']=45
F.loc['MCP_fe','thumb']=90
letters['F']=F


# Function for creatig samples
def create_samples(letter,n_samples,variance):
    # Convert to array and eliminate nan values
    array = letter.as_matrix().ravel()
    array = array[~np.isnan(array)]
    
    # Create samples and add gausian noise
    data = np.tile(array, (n_samples,1))
    noise = np.random.normal(0, variance, data.shape)
    params = data+noise
    
    return params

# Initialize variables
n_samples = 1000
variance = np.arange(5,75,10) # For noise: v degrees in all the angles  
n_letters = len(letters)

params_list = []
labels_list = []

for v in variance:
    
    # Create the parameters structure
    params = np.zeros((n_letters*n_samples,n_params))
    # labels = np.zeros(n_letters*n_samples)
    labels = np.zeros((n_letters*n_samples,n_letters))

    l=0    
    for key, letter in letters.items():

        # Params
        new_params = create_samples(letter,n_samples,v)
        params[(l*n_samples):(l*n_samples+n_samples),:] = new_params

        # Labels
        labels[(l*n_samples):(l*n_samples+n_samples),l]=1
        l+=1

    params_list.append(params)
    labels_list.append(labels)


In [ ]:
# Run the trained feedforward model to generate 6 labels
net = torch.load('twoLayer.pth')
labels_check = net(data_check)

In [ ]:
# Run Generator to generate respective images
# Generator has the following encoding of classes:0-D, 1-C, 2-A, 3-B, 5-E (4-F - not included to analysis)

#Parameters for GAN 
parameters={
'latent_dim': 100, #dimensionality of the latent space
'n_classes': 6, #number of classes for dataset
'img_size': 28, #size of each image dimension
'channels': 1, #number of image channels
}

generator = torch.load('generator_conv_short.pth')

# Sample noise and labels as generator input
z = Variable(FloatTensor(np.random.normal(0, 1, (6, parameters['latent_dim']))))
# labels_check = Variable(LongTensor(np.random.randint(0, parameters['n_classes'], 6)))

# Generate images
generator.eval()
check_imgs = generator(z, labels_check)

fig = plt.figure(figsize=(20,20))
for show in range(6):
    ax = fig.add_subplot(1,6,show+1)
    ax.imshow(check_imgs.numpy()[show,:,:], cmap='gray')
    plt.title(str(labels_check[show].cpu().detach()))